# FILE QA RAG CHATBOT APP

In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install PyMuPDF==1.24.0
!pip install chromadb==0.4.24
!pip install pyngrok==7.1.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.8
    Uninstalling langsmith-0.3.8:
      Successfully uninstalled langsmith-0.3.8
  Attempting uninstall: langchain-core


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:0

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chat_gpt_api.yaml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['OPENAI_API_KEY'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['OPENAI_API_KEY']

In [ ]:
%%writefile app.py
import os
import tempfile
import pandas as pd
from operator import itemgetter

import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma

# ------------------------------
# Page Configuration and Styling
# ------------------------------
st.set_page_config(page_title="RAG-Based QA ChatBOT", page_icon="🤖", layout="wide")
st.markdown(
    """
    <style>
    /* General background */
    body {
        background-color: #f5f5f5;
    }

    /* Sidebar styling */
    .css-1d391kg {
        background-color: #ffffff;
        border-right: 2px solid #e6e6e6;
    }

    /* Chat message styling */
    .chat-message.ai {
        background-color: #e0f7fa;
        color: #006064;
        border-radius: 10px;
        padding: 10px;
        margin-bottom: 5px;
    }
    .chat-message.human {
        background-color: #fce4ec;
        color: #880e4f;
        border-radius: 10px;
        padding: 10px;
        margin-bottom: 5px;
        text-align: right;
    }

    /* Title styling */
    .app-title {
        font-size: 2.5rem;
        font-weight: 700;
        text-align: center;
        margin-bottom: 20px;
    }

    /* Chat container styling */
    .chat-container {
        background-color: #ffffff;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    }
    </style>
    """,
    unsafe_allow_html=True,
)

# ------------------------------
# App Header
# ------------------------------
st.markdown('<div class="app-title">🤖 RAG-Based QA ChatBOT</div>', unsafe_allow_html=True)
st.markdown("### Ask your questions based on your uploaded PDF documents below!")

# ------------------------------
# Sidebar File Upload
# ------------------------------
with st.sidebar:
    st.header("Upload Documents")
    st.markdown("Upload your PDF files that contain the context for the chatbot.")
    uploaded_files = st.file_uploader("Choose PDFs", type=["pdf"], accept_multiple_files=True)

if not uploaded_files:
    st.info("Please upload at least one PDF document to continue.")
    st.stop()

# ------------------------------
# Function to Configure Retriever
# ------------------------------
@st.cache_resource(ttl="1h")
def configure_retriever(uploaded_files):
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploaded_files:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        loader = PyMuPDFLoader(temp_filepath)
        docs.extend(loader.load())

    # Splitting documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    doc_chunks = text_splitter.split_documents(docs)

    # Create embeddings and vector store
    embeddings_model = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(doc_chunks, embeddings_model)
    retriever = vectordb.as_retriever()
    return retriever

# ------------------------------
# Initialize Retriever and LLM
# ------------------------------
retriever = configure_retriever(uploaded_files)
chatgpt = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini", streaming=True)

# ------------------------------
# Define Prompt Template and Chain
# ------------------------------
qa_template = """
Use only the following pieces of context to answer the question at the end.
If you don't know the answer, just say that "The information you are asking for is not available in your files".
Keep the answer as concise as possible.

Context:
{context}

Question: {question}
"""

qa_prompt = ChatPromptTemplate.from_template(qa_template)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

qa_rag_chain = (
    {"context": itemgetter("question") | retriever | format_docs, "question": itemgetter("question")}
    | qa_prompt
    | chatgpt
)

# ------------------------------
# Custom Callback Handlers
# ------------------------------
class StreamlitCallbackHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

class PostMessageHandler(BaseCallbackHandler):
    def __init__(self, container):
        super().__init__()
        self.container = container
        self.sources = []

    def on_retriever_end(self, documents, *, run_id, parent_run_id, **kwargs):
        source_ids = []
        for d in documents:
            metadata = {
                "source": d.metadata.get("source", "N/A"),
                "page": d.metadata.get("page", "N/A"),
                "content": d.page_content[:200]
            }
            idx = (metadata["source"], metadata["page"])
            if idx not in source_ids:
                source_ids.append(idx)
                self.sources.append(metadata)

    def on_llm_end(self, response, *, run_id, parent_run_id, **kwargs):
        if self.sources:
            self.container.markdown("**Sources:**")
            self.container.dataframe(pd.DataFrame(self.sources[:3]), width=1000)

# ------------------------------
# Chat History and UI Display
# ------------------------------
# Using StreamlitChatMessageHistory to persist conversation
streamlit_msg_history = StreamlitChatMessageHistory(key="langchain_messages")
if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("How can I help you?")

# Display past conversation messages in a styled chat container
with st.container():
    st.markdown('<div class="chat-container">', unsafe_allow_html=True)
    for msg in streamlit_msg_history.messages:
        if msg.type == "human":
            st.markdown(f'<div class="chat-message human">{msg.content}</div>', unsafe_allow_html=True)
        else:
            st.markdown(f'<div class="chat-message ai">{msg.content}</div>', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

# ------------------------------
# Chat Input and Processing
# ------------------------------
user_prompt = st.chat_input("Type your question here...")
if user_prompt:
    # Display the human message immediately
    st.markdown(f'<div class="chat-message human">{user_prompt}</div>', unsafe_allow_html=True)

    # Prepare placeholders for streaming response and source info
    with st.container():
        ai_response_placeholder = st.empty()  # For streaming tokens
        sources_container = st.empty()          # For sources info

        # Instantiate our callback handlers (pass the actual instances!)
        stream_handler = StreamlitCallbackHandler(ai_response_placeholder)
        pm_handler = PostMessageHandler(sources_container)

        # Execute the chain using the callback instances
        config = {"callbacks": [stream_handler, pm_handler]}
        response = qa_rag_chain.invoke({"question": user_prompt}, config)

        # Display the final response if not fully streamed using .content attribute
        st.markdown(f'<div class="chat-message ai">{response.content}</div>', unsafe_allow_html=True)

    # Save messages to history using .content attribute
    streamlit_msg_history.add_user_message(user_prompt)
    streamlit_msg_history.add_ai_message(response.content)


Writing app.py


In [ ]:
!streamlit run app.py --server.port 8989 &>./logs.txt &

In [ ]:
from pyngrok import ngrok
import yaml

ngrok.kill()

with open('ngrok_key.yaml', 'r') as file:
    ngrok_token = yaml.safe_load(file)

ngrok.set_auth_token(ngrok_token['ngrok_key'])


ngrok_tunnel = ngrok.connect(8989)
ngrok_tunnel.public_url

'https://27b5-34-56-201-141.ngrok-free.app'